In [1]:
import os
%pwd

'/mnt/cb03386d-9344-47b1-82f9-868fbb64b4ae/python_projects/HIV_inhibitors_classification_and_generation/research'

In [2]:
os.chdir("../")
%pwd

'/mnt/cb03386d-9344-47b1-82f9-868fbb64b4ae/python_projects/HIV_inhibitors_classification_and_generation'

In [3]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataTransformationConfig:
    root_dir: Path
    data_csv: Path
    train_csv: Path
    test_csv: Path
    train_folder: Path
    test_folder: Path
    params: dict
    dataset_val_status: bool

In [4]:
from hivclass.constants import *
from hivclass.utils.main_utils import create_directories, read_yaml

class ConfigurationManager:
    def __init__(
        self,
        config_file_path = CONFIG_FILE_PATH,
        params_file_path = PARAMS_FILE_PATH,
        schema_file_path = SCHEMA_FILE_PATH
    ):
        self.config = read_yaml(config_file_path)
        self.params = read_yaml(params_file_path)
        self.schema = read_yaml(schema_file_path)
        
        create_directories([self.config.artifacts_root])
    
    def get_data_transformation_config(self) -> DataTransformationConfig:
        config = self.config.data_transformation
        params = self.params.data_transformation
        dataset_val_status_file = self.config.data_validation.STATUS_FILE
        
        with open(dataset_val_status_file, 'r') as f:
            status = f.read()
        
        status = bool(str.split(status)[-1])
        
        create_directories([config.root_dir, config.train_folder, config.test_folder])
        
        data_transformation_config = DataTransformationConfig(
            root_dir=config.root_dir,
            data_csv=config.data_csv,
            train_csv=config.train_csv,
            test_csv=config.test_csv,
            train_folder=config.train_folder,
            test_folder=config.test_folder,
            params=params.split_size,
            dataset_val_status=status
        )
        
        return data_transformation_config

In [9]:
from tqdm import tqdm
import pandas as pd
from sklearn.model_selection import train_test_split
import random
import numpy as np
from rdkit import Chem
import torch
from torch_geometric.data import Data
import deepchem as dc

class DataTransformation:
    def __init__(self, config: DataTransformationConfig):
        self.config = config
    
    def test_split_train_balanced(self):
        df = pd.read_csv(self.config.data_csv)
        
        # Separate positive and negative cases
        p_val = df[df.HIV_active == 1].to_numpy()
        n_val = df[df.HIV_active == 0].to_numpy()

        # Ensure class balance by selecting the smaller group as the target
        if len(p_val) >= len(n_val):
            big, small = p_val, n_val
        else:
            big, small = n_val, p_val

        # Stratified test split
        small_train, small_test = train_test_split(
            small,
            test_size=self.config.params,
            random_state=42
        )
        
        big_train, big_test = train_test_split(
            big,
            test_size=(self.config.params * len(small) / len(big)),
            random_state=42
        )

        test = np.concatenate([small_test, big_test])
        
        # Ensure the train set remains balanced by oversampling the smaller class
        train = np.concatenate([
            big_train,
            random.choices(small_train, k=len(big_train) - len(small_train))
        ])

        # Convert back to DataFrame
        train_df = pd.DataFrame(train, columns=df.columns).sample(frac=1, random_state=42)
        train_df.reset_index(drop=True, inplace=True)
        # train_name_column = [f'train_{i+1}' for i in train_df.index]
        # train_df.insert(0, 'name', train_name_column)
        train_df.insert(0, 'name', 'train')
        # train_df.to_csv(self.config.train_csv, index=False)
        
        test_df = pd.DataFrame(test, columns=df.columns).sample(frac=1, random_state=42)
        test_df.reset_index(drop=True, inplace=True)
        # test_name_column = [f'test{i+1}' for i in train_df.index]
        # test_df.insert(0, 'name', test_name_column)
        test_df.insert(0, 'name', 'test')
        # test_df.to_csv(self.config.test_csv, index=False)
        
        return [train_df, test_df]
    
    def data_preparation(self, dfs):
        device = "cuda" if torch.cuda.is_available() else "cpu"

        datas = dfs
        save_dirs = [self.config.train_folder, self.config.test_folder]
        featurizer = dc.feat.MolGraphConvFeaturizer(use_edges=True)

        all_deepchem_data = []

        for data, save_dir in zip(datas, save_dirs):
            # data_df = pd.read_csv(data)
            # name = os.path.splitext(os.path.basename(data))[0]  # Extract filename without extension
            name = data.name[0]
            i = 0
            data_df = pd.DataFrame(columns=['name', 'smiles', 'HIV_active'])
            
            for mol in tqdm(data.itertuples(index=True), total=len(data), desc=f"Processing {name}"):

                mol_obj = Chem.MolFromSmiles(mol.smiles)
                if mol_obj is None:
                    continue  # Skip invalid SMILES strings
                
                label = torch.tensor(mol.HIV_active, dtype=torch.int64, device=device)
                
                graph_features = featurizer._featurize(mol_obj)

                data_deepchem = Data(
                    x=torch.tensor(graph_features.node_features, dtype=torch.float, device=device),
                    edge_attr=torch.tensor(graph_features.edge_features, dtype=torch.float, device=device),
                    edge_index=torch.tensor(graph_features.edge_index, dtype=torch.long, device=device),
                    y=label, smiles=mol.smiles
                )
                all_deepchem_data.append(data_deepchem)

                # Save preprocessed molecule graph
                filename = os.path.join(save_dir, f"{name}_{i}.pt")
                torch.save(data_deepchem, filename)
                
                if os.path.exists(filename):
                    mol_graph = torch.load(filename, weights_only=False)
                    
                    if mol_graph.y.item() != None:
                        data_df.loc[i] = [f"{name}_{i}.pt", mol_graph.smiles, mol_graph.y.item()]
                        i += 1
            
            data_df.to_csv(os.path.join(save_dir, f"{name}.csv"), index=False)
    
    def transformation_compose(self):
        dfs = self.test_split_train_balanced()
        self.data_preparation(dfs)

In [10]:
try:
    config = ConfigurationManager()
    data_transformation_config = config.get_data_transformation_config()
    data_transformation = DataTransformation(config=data_transformation_config)
    results_rdkit, results_deepchem = data_transformation.transformation_compose()

except Exception as e:
    raise e

[2025-04-07 19:28:50,059: INFO: main_utils: created directory at: artifacts]
[2025-04-07 19:28:50,061: INFO: main_utils: created directory at: artifacts/data_transformation]
[2025-04-07 19:28:50,061: INFO: main_utils: created directory at: artifacts/data_transformation/train]
[2025-04-07 19:28:50,062: INFO: main_utils: created directory at: artifacts/data_transformation/test]


Processing train:   0%|          | 0/77636 [00:00<?, ?it/s]

[2025-04-07 19:28:50,287: INFO: molecule_feature_utils: input S not in allowable set ['SP', 'SP2', 'SP3']:]


Processing train:   0%|          | 4/77636 [00:00<37:36, 34.40it/s]

[2025-04-07 19:28:50,364: INFO: molecule_feature_utils: input S not in allowable set ['SP', 'SP2', 'SP3']:]


...

[2025-04-07 19:29:40,313: INFO: molecule_feature_utils: input S not in allowable set ['SP', 'SP2', 'SP3']:]
[2025-04-07 19:29:40,405: INFO: molecule_feature_utils: input UNSPECIFIED not in allowable set ['SP', 'SP2', 'SP3']:]
[2025-04-07 19:29:40,406: INFO: molecule_feature_utils: input UNSPECIFIED not in allowable set ['SP', 'SP2', 'SP3']:]


Processing train:   4%|▍         | 2943/77636 [00:50<20:27, 60.84it/s]

[2025-04-07 19:29:40,563: INFO: molecule_feature_utils: input S not in allowable set ['SP', 'SP2', 'SP3']:]


Processing train:   4%|▍         | 2949/77636 [00:50<21:19, 58.38it/s]


KeyboardInterrupt: 